In [8]:
import time
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# 检查 CUDA 是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义模型 (一个简单的卷积神经网络)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# 加载 CIFAR-10 数据集
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# 定义训练函数
def train_model(model, trainloader, device, epochs=10, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # Use Adam optimizer

    start_time = time.time()
    for epoch in range(epochs):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data[0].to(device), data[1].to(device) # Move data to device

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

    end_time = time.time()
    training_time = end_time - start_time
    print('Finished Training')
    return training_time


def test_model(model, testloader, device):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


# 创建模型并移至 CPU 或 GPU
cpu_model = Net().to("cpu")
gpu_model = Net().to(device)

# 在 CPU 上训练
cpu_start_time = time.time()
cpu_time = train_model(cpu_model, trainloader, "cpu", epochs=10, learning_rate=0.001)
cpu_accuracy = test_model(cpu_model, trainloader, "cpu")

print(f"CPU Training Time: {cpu_time:.4f} seconds, Accuracy: {cpu_accuracy:.2f}%")


# 在 GPU 上训练
gpu_start_time = time.time()
gpu_time = train_model(gpu_model, trainloader, device, epochs=10, learning_rate=0.001)
gpu_accuracy = test_model(gpu_model, trainloader, device)

print(f"GPU Training Time: {gpu_time:.4f} seconds, Accuracy: {gpu_accuracy:.2f}%")

Using device: cuda:0
Files already downloaded and verified
Finished Training
CPU Training Time: 190.9023 seconds, Accuracy: 70.63%
Finished Training
GPU Training Time: 139.1529 seconds, Accuracy: 69.36%


In [4]:
model=Net()
def count_parameters(model):
    """Counts the total number of trainable parameters in a PyTorch model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


total_params = count_parameters(model)
print(f"Total number of trainable parameters: {total_params}")

Total number of trainable parameters: 62006
